In [1]:
import numpy as np
import pandas as pd


In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [7]:
df= pd.read_csv('covid_toy.csv')

In [8]:
df.head()

age  gender  fever cough     city has_covid
0   60    Male  103.0  Mild  Kolkata        No
1   27    Male  100.0  Mild    Delhi       Yes
2   42    Male  101.0  Mild    Delhi        No
3   31  Female   98.0  Mild  Kolkata        No
4   65  Female  101.0  Mild   Mumbai        No

In [10]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [12]:
df['fever'].isnull().sum()

10

In [13]:
#Fever - simple imputer
#Cough - Ordinalcencoder
#gender, city - Onehot encoder
#has_covid - label encoder

# test train split

In [14]:
from sklearn.model_selection import train_test_split

In [19]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

In [20]:
x_train

age  gender  fever   cough       city
32   34  Female  101.0  Strong      Delhi
31   83    Male  103.0    Mild    Kolkata
74   34  Female  104.0  Strong      Delhi
40   49  Female  102.0    Mild      Delhi
55   81  Female  101.0    Mild     Mumbai
..  ...     ...    ...     ...        ...
38   49  Female  101.0    Mild      Delhi
69   73  Female  103.0    Mild      Delhi
62   56  Female  104.0  Strong  Bangalore
79   48  Female  103.0    Mild    Kolkata
36   38  Female  101.0    Mild  Bangalore

[80 rows x 5 columns]

In [21]:
y_train

32    Yes
31     No
74     No
40     No
55    Yes
     ... 
38    Yes
69     No
62    Yes
79    Yes
36     No
Name: has_covid, Length: 80, dtype: object

# Without columns transformer (Aam Zindagi)

In [80]:
#simple imputer -> fever
si= SimpleImputer()

x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.fit_transform(x_test[['fever']])


x_train_fever
x_train_fever.shape

(80, 1)

In [77]:
#OrdinalEncoder  -> Cough
oe= OrdinalEncoder(categories = [['Mild', 'Strong']])

x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])


x_train_cough
x_train_cough.shape

(80, 1)

In [78]:
#OneHotEncoder -> Gender, City
ohe= OneHotEncoder(drop='first', sparse=False)

x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])
x_test_gender_city = ohe.fit_transform(x_test[['gender', 'city']])


x_train_gender_city
x_train_gender_city.shape

(80, 4)

In [79]:
# Extracting Age

x_train_age = x_train.drop(columns= ['gender', 'city', 'cough', 'fever']).values
x_test_age = x_test.drop(columns= ['gender', 'city', 'cough', 'fever']).values

x_train_age
x_train_age.shape

(80, 1)

In [53]:
x_train_transofrmed = np.concatenate((x_train_age, x_train_gender_city, x_train_fever, x_train_cough), axis=1)
x_test_transofrmed = np.concatenate((x_test_age, x_test_gender_city, x_test_fever, x_test_cough), axis=1)

In [57]:
x_train_transofrmed.shape

(80, 7)

In [56]:
x_test_transofrmed.shape

(20, 7)

# Column transformer

In [60]:
from sklearn.compose import ColumnTransformer

In [66]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]),['cough']),
    ('tnf3', OneHotEncoder(drop='first', sparse=False), ['gender', 'city'])
    ], remainder = 'passthrough')

In [72]:
df1= pd.DataFrame(transformer.fit_transform(x_train))

In [73]:
df1

0    1    2    3    4    5     6
0   101.0  1.0  0.0  1.0  0.0  0.0  34.0
1   103.0  0.0  1.0  0.0  1.0  0.0  83.0
2   104.0  1.0  0.0  1.0  0.0  0.0  34.0
3   102.0  0.0  0.0  1.0  0.0  0.0  49.0
4   101.0  0.0  0.0  0.0  0.0  1.0  81.0
..    ...  ...  ...  ...  ...  ...   ...
75  101.0  0.0  0.0  1.0  0.0  0.0  49.0
76  103.0  0.0  0.0  1.0  0.0  0.0  73.0
77  104.0  1.0  0.0  0.0  0.0  0.0  56.0
78  103.0  0.0  0.0  0.0  1.0  0.0  48.0
79  101.0  0.0  0.0  0.0  0.0  0.0  38.0

[80 rows x 7 columns]

In [71]:
transformer.transform(x_test).shape

(20, 7)